Buisness Scenario: Build a model that can learn language pattern from text and genertae new sentences ,predict the next words when some words are provided to it.

In [ ]:

pip install nltk

In [ ]:
import nltk
nltk.download('punkt_tab')


In [ ]:
# Import all the necessary libraries
from warnings import filterwarnings
filterwarnings(action='ignore')

# Preprocess the text
from tensorflow.keras.preprocessing.text import Tokenizer # creates work tokens, number sequences
from tensorflow.keras.utils import pad_sequences # padding
from keras.preprocessing import sequence
# Model
from keras.models import Sequential
from keras.layers import Input,Dense,LSTM,Embedding


LOad the dataset

In [ ]:
import pandas as pd
data = pd.read_csv('/content/Shakespeare_data.csv')

print(data.head())

In [ ]:
data[:15]

We are using Keras Tokenizer here. Few points are discussed below related to fit_on_texts method and texts_to_sequences method: tokenizer.fit_on_texts(texts) : Builds the word index (vocabulary) from your dataset.

It looks at the list of texts you give it. It assigns a unique integer index to each unique word. The most frequent word gets index 1, the next most frequent 2, and so on. Index 0 is reserved for padding.

Example: texts = ["The wizard cast a spell", "The spell was powerful"] tokenizer.fit_on_texts(texts) print(tokenizer.word_index) Output: {'the': 1, 'spell': 2, 'wizard': 3, 'cast': 4, 'a': 5, 'was': 6, 'powerful': 7} tokenizer.texts_to_sequences(texts): Converts your texts into sequences of integers based on the word index created by fit_on_texts.

Example: sequences = tokenizer.texts_to_sequences(["The wizard cast a spell"])
print(sequences) Output: [[1, 3, 4, 5, 2]]


In [ ]:
# Intializing the tokenizer
tokenizer = Tokenizer()
# Fit on texts - pass the data
tokenizer.fit_on_texts(data['PlayerLine'])
word_ind = tokenizer.word_index
print(word_ind)




In [ ]:
reverse_word_index = {value:key for key,value in word_ind.items()}
reverse_word_index

Text on sequences

In [ ]:


vocab_size = len(word_ind) + 1
# to reserve for padding

In [ ]:
vocab_size

In [ ]:
input_sequences = []
for line in data['PlayerLine']:   #iterate through each row in the column
  token_list = tokenizer.texts_to_sequences([line])[0]   #tokenize the line

  for i in range(1,len(token_list)):   # for i in range(1,4)
    n_gram_sequence = token_list[:i+1]    # token_list[0:2] = 0th index,1st index
    # token_list[0:3] = 0th index,1st index,2nd index
    # token_list[0:4] = 0th index, 1st index, 2nd index, 3rd index
    input_sequences.append(n_gram_sequence) # [[4,5],[4,5,6],[4,5,6,7]]
print(input_sequences[:15])

padding to bring all the ngram tokens to same size. to bring all the setences to same length

max([90,89,77])
     

In [ ]:

max_length = 32
input_sequences = sequence.pad_sequences(input_sequences,maxlen=max_length,truncating ='post')
input_sequences[:15]



Separate x and y

In [ ]:
x = input_sequences[:,:-1]
y = input_sequences[:,-1]



In [ ]:


x[:10]

In [ ]:
y[:10]

In [ ]:
y.shape


y.shape
     

If you want to use categorical_crossentropy, then convert y into y cateogrical. y_cat = to_categorical(y)




Model Buidling

In [ ]:
from tensorflow.keras.utils import to_categorical
y_categorical = to_categorical(y,num_classes=vocab_size)# converts integer labels to categorical format                                                          #where total length here represents vocabulary size

model = Sequential()
# Provide the input as max length indicating total number of sentences
model.add(Input(shape=(max_length,)))# input shape
# Add the layers
model.add(Embedding(input_dim=vocab_size,output_dim=50,trainable=False)) # Embedding layers to represent words
model.add(LSTM(16,return_sequences=True,dropout=0.4)) # return sequences will provide the sequences to next LSTM layer
model.add(LSTM(8,dropout=0.3)) # LSTM layer for sequential learning
# Add output layer # predicts the probabilities for each word in the vocabulary
model.add(Dense(vocab_size, activation='softmax')) #dense and softmax layer for next word prediction



In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)


In [ ]:
nn = model.fit(x,y_categorical,validation_split=0.2,epochs=10,callbacks=[early_stop], batch_size = 2)


In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
def generate_text(user_text, next_words=50):
    for _ in range(next_words):
        # preparing our user text ready for the model
        token_list = [tokenizer.word_index.get(w, 0) for w in word_tokenize(user_text.lower())]
        token_list = sequence.pad_sequences([token_list],maxlen=max_length-1)
        # give the proceesed text to model for prediction of next 50words
        predicted_probs = model.predict(token_list.reshape(1, max_length-1), verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1) # it gives you the index of next word
        output_word = reverse_word_index.get(predicted[0], '') # convert the index into word using index_word
        user_text += ' ' + output_word
        # user_text = user_text + output_word
    return user_text


In [ ]:
#Example usage
user = "Find we are "
generated_text = generate_text(user, next_words = 75)
print(generated_text)


# New Section